In [41]:

from config import get_config

from CGPT_utils import *

from train import *

from config import get_config
cfg = get_config()
cfg['batch_size'] = 6
cfg['preload'] = None
cfg['num_epochs'] = 30

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(cfg)

found tokenizer already.


c:\Users\Mark\Desktop\AI_projects\CGPT-1\dataset.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataframe['encoder_input'] = self.dataframe[src_lang].apply(lambda x: tokenizer_src.encode(x, padding='max_length', truncation=True, max_length=self.seq_len, return_tensors='pt'))
c:\Users\Mark\Desktop\AI_projects\CGPT-1\dataset.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.dataframe['decoder_input'] = self.dataframe[tgt_format].apply(lambda x: tokenizer_tgt.encode(x, padding='max_length', 

In [42]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"

In [43]:
# Load the latest model checkpoint
latest_checkpoint = "./weights/tdmodel_29.pt"
checkpoint = torch.load(latest_checkpoint)

# Load the model state from the checkpoint
model_state_dict = checkpoint['model_state_dict']

# Initialize the model
model = get_model(cfg, tokenizer_src.vocab_size, tokenizer_tgt.vocab_size).to(device)

# Load the model state
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [44]:
def greedy_decode(model, source, source_mask, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.encode("<s>", add_special_tokens=False)[0]
    # print(sos_idx)
    eos_idx = tokenizer_tgt.encode("</s>", add_special_tokens=False)[0]

    # Initialize the decoder input with the sos token
    decoder_input = torch.cat([torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device), source], dim=1)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

In [45]:

def inference(input_sequence):
    
    input_tokens = tokenizer_tgt.encode(input_sequence, add_special_tokens=False)

    # Prepare Input Tensors
    max_len = 64
    input_tensor = torch.tensor(input_tokens).unsqueeze(0)  # Add batch dimension
    source_mask = causal_mask(input_tensor.size(1)).unsqueeze(0)  # Assuming decoder mask creation is available

    # Query the Model
    output_tensor = greedy_decode(model, input_tensor, source_mask, tokenizer_tgt, max_len, device)

    # Decode Output
    output_sequence = tokenizer_tgt.decode(output_tensor.cpu().numpy())

    print("Input Sequence:", input_sequence)
    print("Model Output:", output_sequence)
    
inference("C1=CC=C(C=C1)C")


Input Sequence: C1=CC=C(C=C1)C
Model Output: <s>C1=CC=C(C=C1)C=O</s>


In [46]:
inference("C1CC1C(=O)NC2=CC=CC(=C2)N")

Input Sequence: C1CC1C(=O)NC2=CC=CC(=C2)N
Model Output: <s>C1CC1C(=O)NC2=CC=CC(=C2)N3=NC=NC(=C3)NC4=
